In [1]:
import os
import sys
path = !echo ${VIRTUAL_ENV}
path = os.path.join(path[0], '..')
sys.path.append(path)
from helper.utils import load_from_pickle, load_from_shelve
import numpy as np
import pandas as pd
import collections
# from plotly import tools
# import plotly as py
# import plotly.graph_objs as go
#import impywidgets as widget
# py.offline.init_notebook_mode(connected=True)

/home/jackalhan/Development/github/more_meaningful_representations/venv35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [99]:
base_data_path = "/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_2_epoch_1_mar_1.0_sf_0.05_0.05_wd_0.001_0.001_lr_0.001_dim_8196_8196_keep_1_1_seed_66_66_loss_v3"

In [106]:
debug_files = {}
for i, structure in enumerate(os.walk(base_data_path)):
    root, dirs, files = structure
    for file in files:
        if file.endswith(".pkl"):            
            debug_files[i] = os.path.join(root, file)
["{} --> {}".format(key, value) for key, value in debug_files.items()]

['0 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_2_epoch_1_mar_1.0_sf_0.05_0.05_wd_0.001_0.001_lr_0.001_dim_8196_8196_keep_1_1_seed_66_66_loss_v3/debug_dict.pkl']

In [107]:
model_indx =0
my_dict = load_from_pickle(os.path.join(base_data_path, 
                                        debug_files[model_indx]))
num_of_epochs = my_dict['epochs']

In [108]:
eval_metrics = ['recall', 'precision', 'map', 'dcg', 'ndcg', 'arp', 'mrp']
top_k = ['eval_metric_name', '1', '2', '3', '5', '10', '20', '50', 'all']

#extract feature names and k into the hiarchical format
eval_metrics = {}
for key, value in my_dict['1'].items():
    _ = key.split('_')
    fn_name = _[0]
    k = _[1]
    if fn_name not in eval_metrics:
        eval_metrics[fn_name]={}
    if k == 'all':
        eval_metrics[fn_name][1000] = value
    else:
        eval_metrics[fn_name][int(k)] = value

# now place them to the dataframes
dataset = []
for fn_name, ks in eval_metrics.items():
    ks_ = collections.OrderedDict(sorted(ks.items()))
    row = []
    for k in top_k:
        if k == 'eval_metric_name':
            row.append(fn_name)
            continue
        if k == 'all':
            _k = 1000
        else:
            _k = int(k)
        if _k in ks_:
            row.append(ks[int(_k)])
        else:
            row.append('-')
    dataset.append(row)
    #print(fn_name)
df_dataset = pd.DataFrame(data=dataset, columns=top_k)
df_dataset

,eval_metric_name,1,2,3,5,10,20,50,all
0,map,0.453398,0.516019,0.535113,0.549822,0.559959,0.564501,0.566202,-
1,recall,0.453398,0.578641,0.635922,0.7,0.774757,0.839806,0.891262,-
2,mrp,-,-,-,-,-,-,-,0.566902
3,precision,0.453398,0.28932,0.211974,0.14,0.0774757,0.0419903,0.0178252,-
4,dcg,0.453398,0.532417,0.561058,0.587506,0.611833,0.628317,0.638624,-
5,arp,-,-,-,-,-,-,-,36.7126
6,ndcg,0.453398,0.532417,0.561058,0.587506,0.611833,0.628317,0.638624,-


In [64]:
my_dict['1'].items()

dict_items([('ndcg_3', 0.37390348), ('map_2', 0.35333333333333333), ('dcg_2', 0.36040348), ('dcg_5', 0.38744393), ('recall_5', 0.44033333333333335), ('ndcg_10', 0.40131763), ('map_10', 0.3755247354497353), ('ndcg_50', 0.42750308), ('precision_10', 0.04833333333333333), ('recall_10', 0.48333333333333334), ('map_20', 0.37906500779635455), ('precision_3', 0.13577777777777778), ('map_5', 0.3698166666666666), ('precision_20', 0.02666666666666667), ('recall_1', 0.3263333333333333), ('map_3', 0.3623333333333334), ('ndcg_20', 0.4140497), ('dcg_1', 0.32633334), ('mrp_all', 0.3829111), ('recall_2', 0.38033333333333336), ('ndcg_2', 0.36040348), ('precision_2', 0.19016666666666668), ('ndcg_5', 0.38744393), ('recall_20', 0.5333333333333333), ('precision_50', 0.012013333333333334), ('dcg_50', 0.42750308), ('ndcg_1', 0.32633334), ('dcg_20', 0.4140497), ('dcg_10', 0.40131763), ('precision_5', 0.08806666666666667), ('map_1', 0.3263333333333333), ('recall_3', 0.4073333333333333), ('dcg_3', 0.37390348), 